In [1]:
from PIL import Image
import numpy as np

In [2]:
im = Image.open('/Users/sean/Downloads/spotted-lanternfly-nymphs-adults.jpg', 'r')
pix_val = np.array(im.getdata())

In [3]:
print(type(pix_val))
print(pix_val.shape)
print(im.size)

<class 'numpy.ndarray'>
(349700, 3)
(650, 538)


In [4]:
colrow = pix_val.reshape(im.size[0],im.size[1],3)

In [5]:
print(colrow.shape)

(650, 538, 3)


In [6]:
# This is a single column
print(colrow[0][0])
def calc_lum(a):
    # Assumes a is of shape (3,)
    return np.sqrt(0.299*a[0]**2 + 0.587*a[1]**2 + 0.114*a[2]**2)

[238 211 190]


In [7]:
# https://stackoverflow.com/questions/1903462/how-can-i-zip-sort-parallel-numpy-arrays
# I want to do something like this, generate an array (a) which can be easily sorted,
# then use the sort order to sort the original array (b) by the same method.
# Here, array (b) represents a single column of pixel data (r g b).
b = np.array([[1,2,3], [4,5,6], [7,8,9], [1,1,1], [2,2,2], [3,3,3]])
print(b.shape)
a = np.array([6,3,4,2,1,5])
print(a.argsort())
print(b[a.argsort()])

# Array (a) is calculated by manipulating and then summing the entries of array (b).
# This is difficult to do all at once, but we can manipulate each column of the Nx3 array
# and then sum the result along axis=1
b[:,0] = b[:,0] * 2
print(b)
print(b.sum(axis=1))

(6, 3)
[4 3 1 2 5 0]
[[2 2 2]
 [1 1 1]
 [4 5 6]
 [7 8 9]
 [3 3 3]
 [1 2 3]]
[[ 2  2  3]
 [ 8  5  6]
 [14  8  9]
 [ 2  1  1]
 [ 4  2  2]
 [ 6  3  3]]
[ 7 19 31  4  8 12]


In [15]:
from PIL import Image
import random, sys
from time import time

def quicker_sort(pixels):
    pixels = np.array(pixels)
    pix_cpy = pixels.copy()
    pix_cpy = pix_cpy.sum(axis=1)
    return pixels[pix_cpy.argsort()]

def quick_sort(pixels):
    #Quicksort function that sorts pixels based on combined RGB values (R + B + G)
    if pixels == []:
        return pixels

    else:
        pivot = pixels[0]
        lesser = quick_sort([x for x in pixels[1:] if (x[0] + x[1] + x[2]) < (pivot[0] + pivot[1] + pivot[2])])
        greater = quick_sort([x for x in pixels[1:] if (x[0] + x[1] + x[2]) >= (pivot[0] + pivot[1] + pivot[2])])
        return lesser + [pivot] + greater

# def quick_sort_arr(pixels)
#     if pixels == np.array([]):
#         return pixels
#     else:
#         pivot = pixels[0]
#         lesser 
 
def sort_all_pixels(image):
    #sorts every line of pixels
    print("Sorting all pixels.")
    
    print("Opening image...")
    tick = time()
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)
    tock = time()
    print("Time to load image: ", tock-tick)
    
    tick = time()
    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])
    tock = time()
    print("Time to get pixels: ", tock-tick)

    tick = time()
    print("Vectorized sorting pixels...")
    for y in range(img.size[1]):
        sorted_pixels.append(quicker_sort(pixels[y]))
    tock = time()
    print("Time to vectorized sort: ", tock-tick)
    
    tick = time()
    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            #print(type(sorted_pixels[y][x]), "done")
            new.putpixel((x, y), tuple(sorted_pixels[y][x])) #apply the pixels to the new image
    tock = time()
    print("Time to place pixels: ", tock-tick)
    print("Saving image...")
    new.save('output-sortall-vectorized.png')

tick = time()
sys.setrecursionlimit(10000) #Increase the recursion depth limit. Without this, the script fails on larger images because quicksort recurses too much.
sort_all_pixels("/Users/sean/Downloads/IMG_4783.jpg")
tock = time()
print("{:.2f} seconds".format(tock-tick))

Sorting all pixels.
Opening image...
Get data...
Time to load image:  0.02558612823486328
Getting pixels...
Time to get pixels:  0.20251226425170898
Vectorized sorting pixels...
Time to vectorized sort:  0.7173688411712646
Placing pixels...
Time to place pixels:  1.5200660228729248
Saving image...
2.74 seconds


In [16]:
1100*609

669900

In [ ]:
def sort_all_pixels(image):
    #sorts every line of pixels
    print("Sorting all pixels.")
    
    print("Opening image...")
    tick = time()
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)
    tock = time()
    print("Time to load image: ", tock-tick)
    
    tick = time()
    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])
    tock = time()
    print("Time to get pixels: ", tock-tick)

    tick = time()
    print("Recursively sorting pixels...")
    for y in range(img.size[1]):
        sorted_pixels.append(quick_sort(pixels[y]))
    tock = time()
    print("Time to recursive sort: ", tock-tick)
    
    tick = time()
    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            #print(type(sorted_pixels[y][x]), "done")
            new.putpixel((x, y), tuple(sorted_pixels[y][x])) #apply the pixels to the new image
    tock = time()
    print("Time to place pixels: ", tock-tick)
    print("Saving image...")
    new.save('output-sortall-recursive.png')
    
def random_sort_pixels(image, intensity):
    #sorts pixels in a random fashion

    if intensity > 100:
        intensity = 100

    print("Sorting " + image + " with intensity " + str(intensity))

    #Open the image, convert it to RGBA, get the pixels 
    print("Opening image...")
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)

    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])

    print("Quicksorting pixels...")
    for y in range(img.size[1]):
        #pick different starting points for each line
        if(random.randint(0, 100) > intensity):
            sorted_pixels.append(pixels[y]) #Don't sort this line of pixels
        else:
            minsort = random.randint(3, len(pixels[y]) - 3) #pick the start of the sorted area on this pixel line
            maxsort = random.randint(minsort, len(pixels[y]) - 1)# pick the end of the sorted area on this pixel line
            sort = []
            for x in range(minsort, maxsort):
                sort.append(pixels[y][x]) 

            sort = quick_sort(sort) #sort the pixels by brightness

            i = 0
            for x in range(minsort, maxsort):
                pixels[y][x] = sort[i]
                i = i + 1

            sorted_pixels.append(pixels[y])


    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            new.putpixel((x, y), sorted_pixels[y][x]) #apply the pixels to the new image

    print("Saving image...")
    new.save('output-sortrandom.png')

def sort_pixels_pivot(image):
    print ("Sorting pixels on pivot.")

    #Open the image, convert it to RGBA, get the pixels 
    print("Opening image...")
    img = Image.open(image)
    img = img.convert('RGBA')
    print("Get data...")
    data = img.load()

    new = Image.new('RGBA', img.size)

    pixels = []
    sorted_pixels = []
    print("Getting pixels...")
    #Load all of the pixels into the pixels list
    for y in range(img.size[1]):
        pixels.append([])
        for x in range(img.size[0]):
            pixels[y].append(data[x, y])

    print("Quicksorting pixels...")

    minsort = random.randint(3, img.size[0] - 3) #get sorting pivot
    for y in range(img.size[1]):
        maxsort = random.randint(minsort, len(pixels[y]) - 1) #pick the end of the sorted area on this pixel line
        sort = []
        for x in range(minsort, maxsort):
            sort.append(pixels[y][x])
        sort = quick_sort(sort) #sort the pixels by brightness

        i = 0
        for x in range(minsort, maxsort):
            pixels[y][x] = sort[i]
            i = i + 1

        sorted_pixels.append(pixels[y])


    print("Placing pixels...")
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            new.putpixel((x, y), sorted_pixels[y][x]) #apply the pixels to the new image

    print("Saving image...")
    new.save('output-sortpivot.png')
    
random_sort_pixels("/Users/sean/Downloads/spotted-lanternfly-nymphs-adults.jpg", 100)
sort_pixels_pivot("/Users/sean/Downloads/spotted-lanternfly-nymphs-adults.jpg")